# Hello Hybrid Jobs with CUDA-Q

[CUDA-Q](https://nvidia.github.io/cuda-quantum/latest/index.html) offers a unified programming model designed for a hybrid setting for CPUs and GPUs. In this notebook, you will learn how to run Amazon Braket Hybrid Jobs with CUDA-Q using Bring Your Own Container (BYOC). BYOC enables you to configure the environment that you want to use in your jobs. This notebook assumes basic knowledge of Amazon Braket Hybrid Jobs. You can learn about Hybrid Jobs from [this page](https://docs.aws.amazon.com/braket/latest/developerguide/braket-what-is-hybrid-job.html) in the Amazon Braket Developer Guide. 

First, a docker container is built with CUDA-Q and other GPU related settings configured. The procedure for BYOC is presented in this page from [Braker Developer Guide](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-byoc.html). The required files for building a container with CUDA-Q are in the "container" folder, including
- Dockerfile: Describes how the container is built.
- requirements.txt: Additional Python dependencies to include.
- braket_container.py: The start-up script of a job container (optional).

These files include basic settings to use CUDA-Q in jobs. You can modify these files to suit your need. For example, you can add more Python dependencies to "requirements.txt" and other dependencies to "Dockerfile".

In addition, there is a shell script "container_build_and_push.sh" that automates the procedure of pulling, building and pushing container images. The shell script requires Docker to be installed on your local machine. If you don't have Docker already install, you can follow the [instruction in this page](https://www.docker.com/) to install it. The shell script takes two parameters:
- image-name
- region-name

The shell script can be called with the following syntax:
```
container/container_build_and_push.sh <image-name> <region-name>
```
When it's called, it will build a docker container and upload the container image to your [ECR](https://aws.amazon.com/ecr/) repository. The container image will persist in your ECR repository until you update it. The arn of the image will take the format: `<aws-account-id>.dkr.ecr.<region-name>.amazonaws.com/<container-image-name>:latest`.

In [ ]:
! container/container_build_and_push.sh braket-cudaq-byoc-job us-west-2

## Hybrid job with BYOC
Now, we have prepared the environment for CUDA-Q in a container image. Let's run a BYOC job! First, we start with the necessary imports.

In [1]:
import numpy as np

from braket.jobs import hybrid_job
from braket.jobs.config import InstanceConfig
from braket.jobs.environment_variables import get_job_device_arn

We also prepare the URI of the container image. Fill the proper value of `aws_account_id`, `region_name` and `container_image_name` in the cell below. For example, with the shell command above, `region_name="us-west-2"` and `container_image_name="braket-cudaq-byoc-job"`. The cell below prints out the image URI.

In [7]:
aws_account_id = "<aws-account-id>"
region_name = "<region-name>"
container_image_name = "<container-image-name>"

image_uri = f"{aws_account_id}.dkr.ecr.{region_name}.amazonaws.com/{container_image_name}:latest"
print(image_uri)

<aws-account-id>.dkr.ecr.<region-name>.amazonaws.com/<container-image-name>:latest


## Local job with CUDA-Q
Before submitting a job to AWS, it is recommended to start with a local job. A local job runs scripts in a container locally, using the local computing resource. It is a good way to test your code with a small problem size before scaling up.

Here, let's use Bell circuit with CUDA-Q for the local job. This example does not require a GPU to run. The `hello_quantum` function in the code snippet below defines an experiment to sample a Bell circuit. The string `qpp-cpu` in the `device` keyword argument of the decorator is the name of a CUDA-Q CPU simulator. You can view the tutorial of CUDA-Q and the available backends in the [CUDA-Q documentation](https://nvidia.github.io/cuda-quantum/latest/index.html).

In [3]:
@hybrid_job(device='local:nvidia/qpp-cpu', image_uri=image_uri, local=True)
def hello_quantum():
    import cudaq

    # define the backend
    device=get_job_device_arn()
    cudaq.set_target(device.split('/')[-1])
    print("CUDA-Q backend: ", cudaq.get_target())

    # define the Bell circuit
    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(2)
    kernel.h(qubits[0])
    kernel.cx(qubits[0], qubits[1])

    # sample the Bell circuit
    result = cudaq.sample(kernel, shots_count=1000)
    measurement_probabilities = dict(result.items())
    print("Samples: ", measurement_probabilities)
    
    return measurement_probabilities

Skipping python version validation, make sure versions match between local environment and container.


When called, the decorated `hello_quantum` function starts a local job because of the keyword `local=True` in the hybrid_job decorator. The code inside the `hello_quantum` function will run locally in the environment defined by the container image that is built above.

In [ ]:
hello_quantum()

## AWS jobs with CUDA-Q
By remove the `local=True` keyword argument (or, equivalently, set `local=False`), the job will run remotely on AWS.

In [ ]:
@hybrid_job(device='local:nvidia/qpp-cpu', image_uri=image_uri)
def hello_quantum():
    import cudaq
    
    device=get_job_device_arn()
    cudaq.set_target(device.split('/')[-1])
    print(cudaq.get_target())
    
    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(2)
    kernel.h(qubits[0])
    kernel.cx(qubits[0], qubits[1])
    
    result = cudaq.sample(kernel, shots_count=1000)
    measurement_probabilities = dict(result.items())
    print(measurement_probabilities)
    
    return measurement_probabilities

job = hello_quantum()
print(job.arn)

When called, the decorated `hello_quantum` function will create a Braket hybrid job on AWS, running the code defined in the decorated function with the environment specified by the container image built above. You can view the progress of your job with `job.state()` or in the "Hybrid jobs" tab of the Amazon Braket Console.

In [6]:
result = job.result()
print(result)

{'11': 499, '00': 501}


## Summary
This notebook shows you how to build an container that supports CUDA-Q in a BYOC job. The provided shell script automates the procedure of build a container image, simplifying the procedure to a one-line command. This notebook uses Bell circuit as an example to run a local and a AWS Braket job on a CUDA-Q CPU simulator.